# Import

In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, regularizers, evaluators, trainers, datasets, distributions

# DataSet

In [2]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)

# Evaluator

In [3]:
# k
ks = [5, 10, 50]

# Accuracy
recall = evaluators.RecallEvaluator(test_set, ks)

# Unpopularity
unpopularity = evaluators.UnpopularityEvaluator(test_set, ks)

# F1-Score (Recall-Unpopularity)
f1_score = evaluators.F1ScoreEvaluator(test_set, ks)

# Coverage
coverage = evaluators.CoverageEvaluator(test_set, ks)

# Hubness
hubness = evaluators.HubnessEvaluator(test_set, ks)

# Sampler

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

# Model

In [5]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 50
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = True

# distributiuons
dist1 = distributions.Empirical()
dist2 = distributions.Gaussian()
dist3 = distributions.Gamma()

# models
model0 = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
model1 = models.MutualProximityCML(n_user, n_item, dist1, n_dim, n_item_sample, n_user_sample).to(device)
model2 = models.MutualProximityCML(n_user, n_item, dist2, n_dim, n_item_sample, n_user_sample).to(device)
model3 = models.MutualProximityCML(n_user, n_item, dist3, n_dim, n_item_sample, n_user_sample).to(device)

# learning late optimizer
optimizer0 = optim.Adam(model0.parameters(), lr=lr)
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer0 = trainers.BaseTrainer(model0, optimizer0, criterion, sampler, no_progressbar)
trainer1 = trainers.BaseTrainer(model1, optimizer1, criterion, sampler, no_progressbar)
trainer2 = trainers.BaseTrainer(model2, optimizer2, criterion, sampler, no_progressbar)
trainer3 = trainers.BaseTrainer(model3, optimizer3, criterion, sampler, no_progressbar)

# Recall

In [6]:
# train0
trainer0.fit(n_batch, n_epoch, recall, valid_per_epoch)

In [7]:
# train3
trainer3.fit(n_batch, n_epoch, recall, valid_per_epoch)

In [8]:
# CML
trainer0.valid_scores

,Recall@5,Recall@10,Recall@50,epoch,losses
0,0.024178,0.045491,0.220621,0,NaN
0,0.068695,0.164495,0.599927,10,0.471252
0,0.205409,0.335468,0.693551,20,0.302992
0,0.254930,0.395895,0.736429,30,0.258581
0,0.276803,0.414089,0.751590,40,0.247828
0,0.283693,0.428971,0.762588,50,0.236332


In [9]:
# MPCML
trainer3.valid_scores

,Recall@5,Recall@10,Recall@50,epoch,losses
0,0.023771,0.044100,0.224758,0,NaN
0,0.034684,0.073519,0.412869,10,0.468369
0,0.127036,0.239308,0.606317,20,0.295796
0,0.175331,0.295733,0.654436,30,0.256107
0,0.189891,0.313664,0.669141,40,0.243503
0,0.206497,0.321491,0.680281,50,0.236839


# Unpopularity

In [10]:
# train0
trainer0.fit(n_batch, n_epoch, unpopularity, valid_per_epoch)

In [11]:
# train3
trainer3.fit(n_batch, n_epoch, unpopularity, valid_per_epoch)

In [12]:
# CML
trainer0.valid_scores

,Unpopularity@5,Unpopularity@10,Unpopularity@50,epoch,losses
0,0.003724,0.011857,0.139126,0,NaN
0,0.003832,0.011998,0.138413,10,0.234547
0,0.004073,0.012260,0.138781,20,0.230923
0,0.004251,0.012310,0.138488,30,0.228357
0,0.004261,0.012338,0.138763,40,0.227455
0,0.004332,0.012576,0.139163,50,0.226170


In [13]:
# MPCML
trainer3.valid_scores

,Unpopularity@5,Unpopularity@10,Unpopularity@50,epoch,losses
0,0.012348,0.028158,0.183710,0,NaN
0,0.012347,0.028288,0.184143,10,0.234458
0,0.012241,0.028594,0.184313,20,0.232117
0,0.012159,0.028672,0.184796,30,0.229655
0,0.012155,0.028525,0.184562,40,0.228865
0,0.012646,0.028809,0.185499,50,0.226187


# F1-sorce

In [14]:
# train0
trainer0.fit(n_batch, n_epoch, f1_score, valid_per_epoch)

In [15]:
# train3
trainer3.fit(n_batch, n_epoch, f1_score, valid_per_epoch)

In [16]:
# CML
trainer0.valid_scores

,F1-score@5,F1-score@10,F1-score@50,epoch,losses
0,0.007664,0.023177,0.207467,0,NaN
0,0.007543,0.023981,0.207280,10,0.227361
0,0.007616,0.023807,0.206290,20,0.226160
0,0.007803,0.023857,0.207063,30,0.227062
0,0.007862,0.023596,0.206317,40,0.224930
0,0.007777,0.023522,0.206609,50,0.224523


In [17]:
# MPCML
trainer3.valid_scores

,F1-score@5,F1-score@10,F1-score@50,epoch,losses
0,0.016928,0.046277,0.264841,0,NaN
0,0.017228,0.046560,0.265271,10,0.226630
0,0.018173,0.046538,0.265818,20,0.224798
0,0.018508,0.046722,0.266324,30,0.224187
0,0.017487,0.046959,0.266888,40,0.222776
0,0.018720,0.047257,0.266019,50,0.221397


# Coverage

In [18]:
# train0
trainer0.fit(n_batch, n_epoch, coverage, valid_per_epoch)

In [19]:
# train3
trainer3.fit(n_batch, n_epoch, coverage, valid_per_epoch)

In [20]:
# CML
trainer0.valid_scores

,coverage@5,coverage@10,coverage@50,epoch,losses
0,0.365584,0.465791,0.650311,0,NaN
0,0.364893,0.468556,0.646856,10,0.224617
0,0.365584,0.467865,0.648929,20,0.223832
0,0.370422,0.467173,0.648238,30,0.223049
0,0.364893,0.463027,0.648238,40,0.223822
0,0.369039,0.467173,0.650311,50,0.222853


In [21]:
# MPCML
trainer3.valid_scores

,coverage@5,coverage@10,coverage@50,epoch,losses
0,0.400829,0.488597,0.664824,0,NaN
0,0.407049,0.494817,0.664824,10,0.223702
0,0.399447,0.494126,0.660677,20,0.222840
0,0.408431,0.486524,0.655840,30,0.222643
0,0.410504,0.487906,0.664133,40,0.224808
0,0.405667,0.498272,0.662059,50,0.222756


# Hubness 

In [22]:
# train0
trainer0.fit(n_batch, n_epoch, hubness, valid_per_epoch)

In [23]:
# train3
trainer3.fit(n_batch, n_epoch, hubness, valid_per_epoch)

In [24]:
# CML
trainer0.valid_scores

,hubness@5,hubness@10,hubness@50,epoch,losses
0,7.758143,7.624306,3.746958,0,NaN
0,7.796405,7.663681,3.754555,10,0.223330
0,7.799108,7.677076,3.740463,20,0.224788
0,7.807956,7.695854,3.749341,30,0.221935
0,7.871783,7.735982,3.758146,40,0.224688
0,7.780044,7.688562,3.742610,50,0.222000


In [25]:
# MPCML
trainer3.valid_scores

,hubness@5,hubness@10,hubness@50,epoch,losses
0,7.285082,6.839575,3.350861,0,NaN
0,7.196412,6.805711,3.363574,10,0.221685
0,7.251024,6.979063,3.368696,20,0.222977
0,7.150867,6.913954,3.353275,30,0.221374
0,7.217352,6.876462,3.372398,40,0.221679
0,7.053827,6.852306,3.380050,50,0.221504
